In [124]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from geopy.geocoders import Nominatim
import geopandas
import geopy

In [125]:
options = Options()
options.add_argument('disable_infobars')

# Start the WebDriver and load the page
wd = webdriver.Chrome(options=chrome_options)
wd.delete_all_cookies()
wd.set_page_load_timeout(30)

wd.get('http://www.dunmire.net/search/?location=&state=Colorado&submit=&move-in-specials=0')

html_page = wd.page_source
soup = BeautifulSoup(html_page, 'html.parser')
last_height = 0

html_page = wd.page_source
soup2 = BeautifulSoup(html_page, 'html.parser')

for element in soup2.body:
    soup.body.append(element) 
wd.quit()


In [126]:
soup2 = BeautifulSoup(html_page, 'html.parser')
results = soup2.find_all('div', attrs={'__75 smallText full_mobile_width'})
print('Total properties impacted: ', len(results))

Total properties impacted:  25


In [127]:
addresses = []
for address in results:
    addresses.append(address.text.strip())

df = pd.DataFrame(columns=['Address'], data=addresses)

In [128]:
df['location'] = df['Address'].apply(locator.geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [129]:
df

,Address,location,point,latitude,longitude,altitude
0,"2130 East Boulder Street, Colorado Springs, Co...","(2130, East Boulder Street, Knob Hill, Colorad...","(38.83854289818632, -104.78918677122837, 0.0)",38.838543,-104.789187,0.0
1,"4125 Galley Road , Colorado Springs, Colorado,...","(4125, Galley Road, Knob Hill, Colorado Spring...","(38.84700573469388, -104.75063875510205, 0.0)",38.847006,-104.750639,0.0
2,"733 Tia Juana Street, Colorado Springs, Colora...","(733, Tia Juana Street, Knob Hill, Colorado Sp...","(38.84276787755102, -104.7731230612245, 0.0)",38.842768,-104.773123,0.0
3,"3929 East San Miguel Street, Colorado Springs,...","(Columbine Leaf Apartments, East San Miguel St...","(38.850949, -104.75491485145807, 0.0)",38.850949,-104.754915,0.0
4,"2330 E Fountain Blvd, Colorado Springs , Color...","(2330, East Fountain Boulevard, Lowell Neighbo...","(38.817432021586654, -104.7854690787877, 0.0)",38.817432,-104.785469,0.0
5,"3914 North Academy Boulevard, Colorado Springs...","(3914, North Academy Boulevard, Knob Hill, Col...","(38.89031768111659, -104.7571800346622, 0.0)",38.890318,-104.757180,0.0
6,"813 South Circle Drive, Colorado Springs, Colo...","(813, South Circle Drive, Knob Hill, Colorado ...","(38.820413, -104.780422, 0.0)",38.820413,-104.780422,0.0
7,"3165 East Fountain Boulevard, Colorado Springs...","(3165, East Fountain Boulevard, Kelker, Colora...","(38.81523558215258, -104.76809636492382, 0.0)",38.815236,-104.768096,0.0
8,"1303 North Nevada Avenue, Colorado Springs, Co...","(1303, North Nevada Avenue, Lowell Neighborhoo...","(38.85211380031264, -104.82092481065627, 0.0)",38.852114,-104.820925,0.0
9,"1170 S Chelton Road, Colorado Springs, Colorad...","(1170, South Chelton Road, Kelker, Colorado Sp...","(38.81482949272966, -104.76620227652805, 0.0)",38.814829,-104.766202,0.0


In [130]:
%%capture
import folium
df = df.dropna()

#boulder_coords = [40.015, -105.2705]
cos_coords = [38.8339, -104.8214]
map1 = folium.Map(
    location=cos_coords, 
    tiles= 'OpenStreetMap',
    zoom_start=10,
)
    
df.apply(lambda row:folium.CircleMarker(radius = 3, location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)


In [131]:
map1